# Analyzing data from Labus et al. with tascCODA

In [1]:
# Setup
import re

import numpy as np
import pandas as pd
import toyplot
import toytree as tt
import re
import toyplot.svg

import ibs_util_functions as util
import tasccoda.tree_ana as ta

/Users/johannes.ostner/opt/anaconda3/envs/scCODA_3/lib/python3.8/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt


In [2]:
# tascCODA running function
data_dir = '../../../tascCODA_data/applications/labus_IBS/results/'

tax_levels = ["Kingdom", "Phylum", "Class", "Order", "Family", "Genus"]

def run_tree_agg_Labus(
    level,
    result_name,
    data=None,
    pen_args={"lambda_0": 50, "phi": 0, "lambda_1": 5},
    formula = "C(host_disease, Treatment('Healthy'))",
    num_samples = 2000
):
    author = "Labus"
    year = 2017
    references = {
        "Genus": "Bacteria*Bacteroidota*Bacteroidia*Bacteroidales*Rikenellaceae*Alistipes",
        "Family": "Bacteria*Bacteroidota*Bacteroidia*Bacteroidales*Rikenellaceae",
        "Order": "Bacteria*Bacteroidota*Bacteroidia*Bacteroidales",
        "Class": "Bacteria*Bacteroidota*Bacteroidia",
        "Phylum": "Bacteria*Bacteroidota",
    }

    if data is None:
        data = util.agg_ibs_data(author, level)
    n_level = tax_levels.index(level)+1

    for char in data.var.index:
        split = char.split(sep="*")

        for n in range(n_level):
            data.var.loc[char, tax_levels[n]] = "*".join(split[:n+1])

    newick = util.df2newick(data.var.reset_index(drop=True))

    tree = tt.tree(newick, tree_format=1)
    data.uns["phylo_tree"] = tree

    data = data[:, tree.get_tip_labels()]

    model = ta.CompositionalAnalysisTree(
        data=data,
        formula=formula,
        # reference_cell_type=references[level],
        reference_cell_type=references["Genus"],
        reg="scaled_3",
        pen_args=pen_args,
        model="new",
        automatic_reference_absence_threshold=0.2
    )
    result = model.sample_hmc_da(num_results=num_samples, num_burnin=int(num_samples/4))

    eff_lv = util.get_phylo_levels(result.effect_df.reset_index(), level)
    cov_lv = result.effect_df.index.get_level_values("Covariate").to_list()
    eff_lv.insert(0, 'Covariate', cov_lv)

    result.effect_df.index = pd.MultiIndex.from_frame(eff_lv)

    nd = result.node_df.reset_index()
    node_lv = util.get_phylo_levels(nd, level, "Node")
    cov_lv = result.node_df.index.get_level_values("Covariate").to_list()
    node_lv.insert(0, 'Covariate', cov_lv)

    result.node_df.index = pd.MultiIndex.from_frame(node_lv)
    result.node_df["Cell Type"] = np.array(nd.loc[:, "Node"])

    filename = f"../../../tascCODA_data/applications/labus_IBS/results/{author.lower()}_{level.lower()}_"
    result.node_df.to_csv(filename + f"tree_agg_{result_name}.csv")

    return result

We first look at IBS vs. healthy
To get some credible results, we set $\lambda_1 = 1$.

We then get:
- one effect for $\phi = -5$
- two effects for $\phi = 0$
- eight effects for $\phi = 5$

These effects are similar to what scCODA finds.

In [3]:
results = {}

In [30]:
phis = [-5, 0, 5]
lambdas = [1]
formula = "C(host_disease, Treatment('Healthy'))"

for phi in phis:
    for l_1 in lambdas:
        name = f"phi_{phi}_l1_{l_1}"
        pa = {"lambda_0": 50, "phi": phi, "lambda_1": l_1}
        res = run_tree_agg_Labus(
            level="Genus",
            result_name=f"test_phi_{phi}_l1_{l_1}",
            pen_args=pa,
            formula=formula,
            num_samples=20000
        )

        results[(phi, l_1)] = res
        print("")
        print(name)
        print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])


Trying to set attribute `.X` of view, copying.


Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:28<00:00, 95.96it/s] 


MCMC sampling finished. (264.728 sec)
Acceptance rate: 85.8%


Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (261.427 sec)
Acceptance rate: 86.0%


Trying to set attribute `.X` of view, copying.



phi_0_l1_1
Covariate                                          Kingdom   Phylum        Class        Order            Family          Genus          
C(host_disease, Treatment('Healthy'))[T.IBS]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales    Tannerellaceae  Parabacteroides   -0.156
                                                                                                         Bacteroidaceae  Bacteroides       -0.662
                                                             Firmicutes    Clostridia   Oscillospirales  NaN             NaN               -0.232
Name: Final Parameter, dtype: float64
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:33<00:00, 93.56it/s] 


MCMC sampling finished. (270.470 sec)
Acceptance rate: 86.3%

phi_5_l1_1
Covariate                                          Kingdom   Phylum        Class          Order              Family              Genus                
C(host_disease, Treatment('Healthy'))[T.IBS]_node  Bacteria  Bacteroidota  Bacteroidia    Bacteroidales      Tannerellaceae      Parabacteroides         -0.845
                                                                                                             Bacteroidaceae      Bacteroides             -1.001
                                                                                                             Prevotellaceae      Prevotella              -0.413
                                                             Firmicutes    Clostridia     Lachnospirales     Lachnospiraceae     Agathobacter            -0.610
                                                                                          Oscillospirales    Ruminococcaceae     Subdoli

In [5]:
# tree drawing function
def draw_tree(node_df, title="", add_sccoda=True, tip_labels=True, save_path=None, eff_max=None):
    """
    Plots a nice tree from a tascCODA node_df

    :param node_df: DataFrame
        Must have columns ["Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Final Parameter"];
        index must be full taxonomic names, e.g. "Bacteria*Bacteroidota*Bacteroidia*Bacteroidales*Rikenellaceae*Alistipes"
    :param title: string
        Title that is printed at the top of the plot
    :param add_sccoda: bool
        Write genera found to be DA by scCODA in red
    :param tip_labels: bool
        If true, add genus names as tip labels
    :param save_path: string
        If not none, plot will be saved as svg to this file
    :param eff_max: float
        Maximum absolute value of effects for scaling of node size.
        If None, ,aximum absolute value of node_df["Final Parameter"] will be used
    :return:
    """

    # Build dictionary tu use with util.build_fancy_tree
    node_df = node_df.reset_index()
    tree_dat = {}
    node_df["level"] = "Kingdom"
    for l in tax_levels[1:]:
        node_df.loc[pd.notna(node_df.reset_index()[l]), "level"] = l

    for l in tax_levels[1:]:
        tree_dat[l] = node_df.loc[node_df["level"] == l].reset_index()

    # Make toytree object, initialize toyplot canvas
    tree, markers = util.build_fancy_tree(tree_dat)
    canvas = toyplot.Canvas(width=1000, height=1000)

    # Area for tree plot
    ax0 = canvas.cartesian(bounds=(50, 800, 50, 800), padding=0)
    ax0.x.show = False
    ax0.y.show = False

    # Determine max effect size, if necessary
    if eff_max is None:
        eff_max = np.max([np.abs(n.effect) for n in tree.treenode.traverse()])

    # Determine label colors from scCODA results. The reference (hardcoded as "Alistipes" will be colored grey)
    if add_sccoda:
        data_sccoda = util.read_authors_results(["Labus"], data_dir, method="sccoda", add=None)["Genus"]
        label_colors = [
            "darkgrey" if n=="Alistipes"
            else "red" if data_sccoda.loc[data_sccoda["Genus"]==n, "Final Parameter"].tolist()[0] != 0
            else "black" for n in tree.get_tip_labels()
        ]
    else:
        label_colors = ["darkgrey" if n=="Alistipes" else "black" for n in tree.get_tip_labels()]

    # draw tree.
    tree.draw(
        axes=ax0,
        layout='c', # circular layout
        edge_type='p', # rectangular edges
        node_sizes=[(np.abs(n.effect) * 20 / eff_max) + 10 if n.effect != 0 else 0 for n in tree.treenode.traverse()], # node size scales with node feature "effect"
        node_colors=[n.color for n in tree.treenode.traverse()], # node color from node feature "color"
        node_style={
            "stroke": "black",
            "stroke-width": "1"
        },
        width=800,
        height=800,
        tip_labels=tip_labels, # Print tip labels or not
        tip_labels_align=True,
        tip_labels_style={"font-size": "12px"},
        tip_labels_colors=label_colors,
        edge_colors=[tree.idx_dict[x[1]].edge_color for x in tree.get_edges()], # edge color from node feature "edge_color"
        edge_widths=3 # width of tree edges
    )

    # add area for plot title
    ax1 = canvas.cartesian(bounds=(50, 800, 50, 100), padding=0, label=title)
    ax1.x.show=False
    ax1.y.show=False

    # add legend for phylum colors
    canvas.legend(markers,
            bounds=(800, 1000, 275, 375),
            label="Phylum"
    )

    # add legend for effect size
    markers2 = [
        (f"{eff_max}",toyplot.marker.create(shape="o", size=20, mstyle={"fill": "black"})),
        (f"{-1*eff_max}",toyplot.marker.create(shape="o", size=20, mstyle={"fill": "white"}))
    ]
    canvas.legend(markers2,
        bounds=(800, 1000, 425, 525),
        label=f"tascCODA Effects"
    )

    # save plot if desired
    if save_path is not None:
        toyplot.svg.render(canvas, save_path)

Comparing with scCODA, we recover all effects found by scCODA and a few more.

In [23]:
for p in [-5, 0, 5]:
    res = results[(p, 1)]

    draw_tree(res.node_df, "Labus")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te1345a08f1be45d8a7d115e5923a617f"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Labus Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 0.315 -0.315

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta650978a703646d48a3b846484c7a74c"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Labus Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 0.627 -0.627

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tdabbcf5a320843ab90f154f253cc77f3"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Labus Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 0.978 -0.978

## Using other covariates

To get all covariates, and therefore effect sizes, on the same scale, we min-max normalize BMI and age

In [7]:
def normalize(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

In [8]:
data_lab = util.agg_ibs_data("Labus", "Genus")
data_lab.obs["host_age_norm"] = normalize(data_lab.obs["host_age"])
data_lab.obs["bmi_norm"] = normalize(data_lab.obs["host_bmi"])
data_lab.obs

,OTU,host_disease,host_subtype,host_age,host_sex,host_bmi,collection_date,author,sample_type,Collection,host_age_norm,bmi_norm
Sample,,,,,,,,,,,,
SRR5245600,ASV1,Healthy,HC,27,male,27.299999,14-Feb-2014,Labus,stool,1st,0.140625,0.392641
SRR5245601,ASV5,Healthy,HC,28,female,NaN,13-Feb-2014,Labus,stool,1st,0.156250,NaN
SRR5245602,ASV1,Healthy,HC,20,female,NaN,14-Feb-2014,Labus,stool,1st,0.031250,NaN
SRR5245603,ASV1,Healthy,HC,27,female,36.796875,18-Feb-2014,Labus,stool,1st,0.140625,0.810471
SRR5245604,ASV1,IBS,IBS-C,20,female,NaN,13-Feb-2014,Labus,stool,1st,0.031250,NaN
SRR5245605,ASV1,Healthy,HC,32,female,20.318630,07-Feb-2014,Labus,stool,1st,0.218750,0.085485
SRR5245606,ASV1,Healthy,HC,18,female,21.219858,11-Feb-2014,Labus,stool,1st,0.000000,0.125136
SRR5245607,ASV1,Healthy,HC,20,female,22.700001,10-Feb-2014,Labus,stool,1st,0.031250,0.190257
SRR5245608,ASV1,Healthy,HC,20,female,20.908524,23-Jan-2014,Labus,stool,1st,0.031250,0.111438


### IBS subtype

Only for $\phi=5$, we find 10 effects on the genus level.

- For constipation, an increase in Anaerostipes, as well as decreases in Bacteroides, Agathobacter, Ruminococcus, Faecalibacterium.
- For Diarrhea, decreases in Parabacteroides, Bacteroides
- For mixed, an increase in Blautia and decreases in Parabacteroides, Faecalibacterium
- For untetermined, nothing

In [27]:
res_subtypes = {}
formula = "host_subtype"
phis = [-5, 0, 5]
for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"subtype_phi_{p}",
                data=data_lab,
                pen_args=pa,
                formula=formula,
                num_samples=20000
            )
    res_subtypes[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])


Trying to set attribute `.X` of view, copying.


Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:37<00:00, 91.83it/s]


MCMC sampling finished. (276.674 sec)
Acceptance rate: 86.6%


Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (276.700 sec)
Acceptance rate: 85.9%


Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:40<00:00, 90.90it/s]


MCMC sampling finished. (279.581 sec)
Acceptance rate: 86.7%
Covariate                             Kingdom   Phylum        Class        Order                                Family                 Genus           
host_subtype[T.IBS-C]_node            Bacteria  Bacteroidota  Bacteroidia  Bacteroidales                        Bacteroidaceae         Bacteroides        -0.426
                                                Firmicutes    Clostridia   Lachnospirales                       Lachnospiraceae        Anaerostipes        0.438
                                                                                                                                       Agathobacter       -0.819
                                                                           Oscillospirales                      Ruminococcaceae        Ruminococcus       -0.262
                                                                                                                                       Faecalib

### Age

We find nothing, for any value of $\phi$

In [10]:
res_age = {}
formula = "host_age_norm"

for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"age_phi_{p}",
                data=data_lab,
                pen_args=pa,
                formula=formula,
                num_samples=20000
            )
    res_age[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])

Trying to set attribute `.X` of view, copying.


Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:24<00:00, 97.77it/s] 


MCMC sampling finished. (257.997 sec)
Acceptance rate: 86.7%


Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (256.458 sec)
Acceptance rate: 86.0%


Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:25<00:00, 97.38it/s] 


MCMC sampling finished. (257.880 sec)
Acceptance rate: 85.6%
Series([], Name: Final Parameter, dtype: float64)


### Sex

We find nothing, for any value of $\phi$

In [11]:
res_sex = {}
formula = "host_sex"

for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"sex_phi_{p}",
                data=data_lab,
                pen_args=pa,
                formula=formula,
                num_samples=20000
            )
    res_sex[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])


Trying to set attribute `.X` of view, copying.


Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:22<00:00, 98.64it/s] 


MCMC sampling finished. (255.713 sec)
Acceptance rate: 86.6%


Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (254.973 sec)
Acceptance rate: 86.6%


Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:25<00:00, 97.42it/s] 


MCMC sampling finished. (258.162 sec)
Acceptance rate: 86.0%
Series([], Name: Final Parameter, dtype: float64)


### BMI (3 samples left out)

We find nothing, for any value of $\phi$

In [12]:
res_bmi = {}
formula = "bmi_norm"

for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"bmi_phi_{p}",
                data=data_lab[~np.isnan(data_lab.obs["bmi_norm"])],
                pen_args=pa,
                formula=formula,
                num_samples=20000
            )
    res_bmi[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.


Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:17<00:00, 101.36it/s]


MCMC sampling finished. (248.162 sec)
Acceptance rate: 86.8%


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (247.828 sec)
Acceptance rate: 87.1%


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:17<00:00, 101.16it/s]


MCMC sampling finished. (248.509 sec)
Acceptance rate: 85.8%
Series([], Name: Final Parameter, dtype: float64)


### All covariates, additive (3 samples left out)

We find only effects on IBS subtype, which are 6 of the 10 effects found when using only IBS subtype

In [13]:
res_sum_all = {}
formula = "host_sex + host_subtype + bmi_norm + host_age_norm"

for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"sum_all_phi_{p}",
                data=data_lab[~np.isnan(data_lab.obs["bmi_norm"])],
                pen_args=pa,
                formula=formula,
                num_samples=20000
            )
    res_sum_all[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.
/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(


Zero counts encountered in data! Added a pseudocount of 0.5.


100%|██████████| 20000/20000 [03:31<00:00, 94.50it/s]


MCMC sampling finished. (265.943 sec)
Acceptance rate: 86.7%


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (270.008 sec)
Acceptance rate: 85.7%


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:34<00:00, 93.25it/s]


MCMC sampling finished. (270.797 sec)
Acceptance rate: 87.1%
Covariate                   Kingdom   Phylum        Class        Order           Family           Genus          
host_subtype[T.IBS-C]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales   Bacteroidaceae   Bacteroides       -0.604
                                      Firmicutes    Clostridia   Lachnospirales  Lachnospiraceae  Agathobacter      -0.313
host_subtype[T.IBS-D]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales   Tannerellaceae   Parabacteroides   -0.265
                                                                                 Bacteroidaceae   Bacteroides       -1.286
host_subtype[T.IBS-M]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales   Tannerellaceae   Parabacteroides   -0.274
                                      Firmicutes    Clostridia   Lachnospirales  Lachnospiraceae  Blautia            0.797
Name: Final Parameter, dtype: float64


### All covariates except BMI, additive

We find only effects on IBS subtype, which are 6 of the 10 effects found when using only IBS subtype.
5 of 6 are the same as when including BMI

In [14]:
res_sum_nobmi = {}
formula = "host_sex + host_subtype + host_age_norm"

for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"sum_nobmi_phi_{p}",
                data=data_lab,
                pen_args=pa,
                formula=formula,
                num_samples=20000
            )
    res_sum_nobmi[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])

Trying to set attribute `.X` of view, copying.


Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:37<00:00, 91.78it/s]


MCMC sampling finished. (274.751 sec)
Acceptance rate: 86.2%


Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (274.549 sec)
Acceptance rate: 85.8%


Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [03:39<00:00, 91.10it/s]


MCMC sampling finished. (275.540 sec)
Acceptance rate: 88.6%
Covariate                   Kingdom   Phylum        Class        Order           Family           Genus          
host_subtype[T.IBS-C]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales   Bacteroidaceae   Bacteroides       -0.381
                                      Firmicutes    Clostridia   Lachnospirales  Lachnospiraceae  Anaerostipes       0.282
                                                                                                  Agathobacter      -0.573
host_subtype[T.IBS-D]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales   Tannerellaceae   Parabacteroides   -0.263
                                                                                 Bacteroidaceae   Bacteroides       -1.260
host_subtype[T.IBS-M]_node  Bacteria  Firmicutes    Clostridia   Lachnospirales  Lachnospiraceae  Blautia            0.643
Name: Final Parameter, dtype: float64


### All covariates except BMI, multiplicative

We find 3 effects on IBS subtypes, as well as one interaction between sex and IBS-D that strengthens the decrease of Bacteroides

In [87]:
res_prod = {}
formula = "host_sex * host_subtype * host_age_norm * bmi_norm"

for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"prod_phi_{p}",
                data=data_lab[~np.isnan(data_lab.obs["bmi_norm"])],
                pen_args=pa,
                formula=formula,
                num_samples=2000
            )
    res_prod[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])

Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.
/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(


Zero counts encountered in data! Added a pseudocount of 0.5.


100%|██████████| 2000/2000 [00:46<00:00, 42.91it/s]


MCMC sampling finished. (70.615 sec)
Acceptance rate: 85.3%


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (76.155 sec)
Acceptance rate: 87.4%


Trying to set attribute `.uns` of view, copying.
Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 2000/2000 [00:46<00:00, 43.45it/s]


MCMC sampling finished. (61.972 sec)
Acceptance rate: 87.4%
Covariate                                    Kingdom   Phylum        Class        Order          Family          Genus      
host_subtype[T.IBS-D]_node                   Bacteria  Bacteroidota  Bacteroidia  Bacteroidales  Bacteroidaceae  Bacteroides   -0.275
host_sex[T.male]:host_subtype[T.IBS-D]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales  Bacteroidaceae  Bacteroides   -0.400
Name: Final Parameter, dtype: float64


In [86]:
res_prod_nobmi = {}
formula = "host_sex * host_subtype * host_age_norm"

for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"prod_nobmi_phi_{p}",
                data=data_lab,
                pen_args=pa,
                formula=formula,
                num_samples=20000
            )
    res_prod_nobmi[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])

Trying to set attribute `.X` of view, copying.


Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [04:07<00:00, 80.92it/s]


MCMC sampling finished. (317.126 sec)
Acceptance rate: 84.9%


Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (337.003 sec)
Acceptance rate: 85.9%


Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 20000/20000 [04:15<00:00, 78.40it/s]


MCMC sampling finished. (329.225 sec)
Acceptance rate: 85.5%
Covariate                                    Kingdom   Phylum        Class        Order           Family           Genus       
host_subtype[T.IBS-C]_node                   Bacteria  Firmicutes    Clostridia   Lachnospirales  Lachnospiraceae  Agathobacter   -0.252
host_subtype[T.IBS-D]_node                   Bacteria  Bacteroidota  Bacteroidia  Bacteroidales   Bacteroidaceae   Bacteroides    -0.570
host_subtype[T.IBS-M]_node                   Bacteria  Firmicutes    Clostridia   Lachnospirales  Lachnospiraceae  Blautia         0.277
host_sex[T.male]:host_subtype[T.IBS-D]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales   Bacteroidaceae   Bacteroides    -0.628
Name: Final Parameter, dtype: float64


In [85]:
res_prod_nobmi_general = {}
formula = "host_sex * host_disease * host_age_norm"

for p in phis:
    pa = {"lambda_0": 50, "phi": p, "lambda_1": 1}

    res = run_tree_agg_Labus(
                level="Genus",
                result_name=f"prod_nobmi_general_phi_{p}",
                data=data_lab,
                pen_args=pa,
                formula=formula,
                num_samples=2000
            )
    res_prod_nobmi_general[phi] = res
    print(res.node_df.loc[np.abs(res.node_df["Final Parameter"]) != 0, "Final Parameter"])


Trying to set attribute `.X` of view, copying.


Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 2000/2000 [00:26<00:00, 76.60it/s]


MCMC sampling finished. (44.989 sec)
Acceptance rate: 86.6%


Trying to set attribute `.X` of view, copying.


...

MCMC sampling finished. (48.934 sec)
Acceptance rate: 86.6%


Trying to set attribute `.X` of view, copying.


Series([], Name: Final Parameter, dtype: float64)
Zero counts encountered in data! Added a pseudocount of 0.5.


/Users/johannes.ostner/Documents/PhD/tree_aggregation/tree_aggregation/tree_agg_model_sslasso.py:455: UserWarning: This feature is untested and might yield different results than expected. Please use sample_hmc().
  warnings.warn(
100%|██████████| 2000/2000 [00:26<00:00, 74.82it/s]


MCMC sampling finished. (44.939 sec)
Acceptance rate: 90.1%
Covariate                 Kingdom   Phylum        Class        Order            Family           Genus          
host_sex[T.male]_node     Bacteria  Bacteroidota  Bacteroidia  Bacteroidales    Bacteroidaceae   Bacteroides       -0.327
host_disease[T.IBS]_node  Bacteria  Bacteroidota  Bacteroidia  Bacteroidales    Tannerellaceae   Parabacteroides   -0.603
                                                                                Bacteroidaceae   Bacteroides       -0.811
                                    Firmicutes    Clostridia   Lachnospirales   Lachnospiraceae  Agathobacter      -0.324
                                                               Oscillospirales  Ruminococcaceae  NaN               -0.350
Name: Final Parameter, dtype: float64


## Paper figures

In [6]:
all_df = []

ndf_subtype = pd.read_csv(f"../../../tascCODA_data/applications/labus_IBS/results/labus_genus_tree_agg_subtype_phi_5.csv")
all_df.append(ndf_subtype)
max_eff = np.max([np.max(np.abs(x["Final Parameter"])) for x in all_df])
print(max_eff)

for p in [-5, 0, 5]:
    ndf = pd.read_csv(f"../../../tascCODA_data/applications/labus_IBS/results/labus_genus_tree_agg_test_phi_{p}_l1_1.csv")
    all_df.append(ndf)
    draw_tree(
        ndf,
        title=f"Healthy vs. IBS, ={p}",
        eff_max=max_eff,
        tip_labels=False,
        # save_path=f"./IBS_plots/IBS_tree_phi_{p}_notip.svg"
    )
    draw_tree(
        ndf,
        title=f"Healthy vs. IBS, ={p}",
        eff_max=max_eff,
        tip_labels=True,
        # save_path=f"./IBS_plots/IBS_tree_phi_{p}.svg"
    )

subtypes = pd.unique(ndf_subtype["Covariate"])
st_names = [re.findall(r"IBS-[A-Za-z]+", s)[0] for s in subtypes]

for i in range(4):
    draw_tree(
        ndf_subtype[ndf_subtype["Covariate"]==subtypes[i]],
        title=f"Healthy vs. {st_names[i]}, ={p}",
        eff_max=max_eff,
        add_sccoda=False,
        tip_labels=False,
        # save_path=f"./IBS_plots/subtype_tree_{st_names[i]}_notip.svg"
    )

for i in range(4):
    draw_tree(
        ndf_subtype[ndf_subtype["Covariate"]==subtypes[i]],
        title=f"Healthy vs. {st_names[i]}",
        eff_max=max_eff,
        add_sccoda=False,
        tip_labels=True,
        # save_path=f"./IBS_plots/subtype_tree_{st_names[i]}.svg"
    )

1.405


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9d6ef07d22a742b29ef2784ae202b94e"> Healthy vs. IBS, =-5 Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6ea6fd9b2ff14767bcb5c10ad7f33894"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Healthy vs. IBS, =-5 Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tdae3f49d9f7f47fa999e3387c7844fe9"> Healthy vs. IBS, =0 Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t021503291a8d47479d9552853d824080"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Healthy vs. IBS, =0 Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

...

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5f9c93b0df34410793ac575bf2d78d7c"> Healthy vs. IBS-unspecified, =5 Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6e0e735d6ff644aabd5c6bee8cecb973"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Healthy vs. IBS-C Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4d9c655a2a3c4712a3971082b3f96a1a"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Healthy vs. IBS-D Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td003637fd9ee4fce804c49c45bcbd4bb"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Healthy vs. IBS-M Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2120a0ede95a4a0c926bfddfcc345075"> Faecalibacterium Subdoligranulum Ruminococcus Anaerofilum Paludicola UBA1819 Incertae_Sedis Angelakisella Negativibacillus UCG-002 UCG-005 UCG-003 Flavonifractor V9D2013_group Oscillibacter NK4A214_group Intestinimonas Butyricicoccus UCG-008 Blautia Dorea Agathobacter Fusicatenibacter Anaerostipes Lachnospiraceae_NK4A136_group Coprococcus Roseburia Lachnoclostridium Sellimonas Marvinbryantia Tuzzerella Lachnospiraceae_AC2044_group CAG-56 Lachnospira Lachnospiraceae_UCG-003 Eisenbergiella Lachnospiraceae_UCG-010 Lachnospiraceae_UCG-001 Romboutsia Terrisporobacter Intestinibacter Family_XIII_AD3011_group Christensenellaceae_R-7_group Christensenella Monoglobus Clostridium_sensu_stricto_1 Peptococcus Holdemanella Turicibacter Catenisphaera Holdemania Faecalitalea Erysipelotrichaceae_UCG-003 Catenibacterium Erysipelatoclostridium Coprobacillus Lactobacillus Streptococcus Enterococcus Dialister Megasphaera Veillonella Acidaminococcus Phascolarctobacterium Prevotella Prevotellaceae_NK3B31_group Paraprevotella Alloprevotella Prevotellaceae_YAB2003_group Alistipes Rikenellaceae_RC9_gut_group Barnesiella Coprobacter Odoribacter Butyricimonas Bacteroides Parabacteroides Eggerthella Gordonibacter Slackia Senegalimassilia Enterorhabdus Collinsella Atopobium Cronobacter Escherichia/Shigella Enterobacter Yersinia Serratia Parasutterella Cloacibacillus Healthy vs. IBS-unspecified Phylum Synergistota Proteobacteria Actinobacteriota Bacteroidota Firmicutes tascCODA Effects 1.405 -1.405

## Get Latex for supplemental tables

In [4]:
ndfs = []
for p in [-5, 0, 5]:
    ndf = pd.read_csv(f"../../../tascCODA_data/applications/labus_IBS/results/labus_genus_tree_agg_test_phi_{p}_l1_1.csv")
    ndf["phi"] = p
    ndfs.append(ndf)

all_ndf = pd.concat(ndfs)
all_ndf["Condition"] = [re.findall(r"\[.*\]", x)[0].replace("[T.", "").replace("]", "") for x in all_ndf["Covariate"]]
print(all_ndf.loc[all_ndf["Final Parameter"] != 0, ["phi", "Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Final Parameter"]])
print(all_ndf.loc[all_ndf["Final Parameter"] != 0, ["phi", "Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Final Parameter"]].to_latex(index=False))

     phi   Kingdom        Phylum          Class              Order  \
119   -5  Bacteria    Firmicutes            NaN                NaN   
0      0  Bacteria  Bacteroidota    Bacteroidia      Bacteroidales   
1      0  Bacteria  Bacteroidota    Bacteroidia      Bacteroidales   
101    0  Bacteria    Firmicutes     Clostridia    Oscillospirales   
0      5  Bacteria  Bacteroidota    Bacteroidia      Bacteroidales   
1      5  Bacteria  Bacteroidota    Bacteroidia      Bacteroidales   
6      5  Bacteria  Bacteroidota    Bacteroidia      Bacteroidales   
43     5  Bacteria    Firmicutes     Clostridia     Lachnospirales   
58     5  Bacteria    Firmicutes     Clostridia    Oscillospirales   
59     5  Bacteria    Firmicutes     Clostridia    Oscillospirales   
89     5  Bacteria    Firmicutes  Negativicutes  Acidaminococcales   
91     5  Bacteria    Firmicutes     Clostridia    Oscillospirales   

                 Family                  Genus  Final Parameter  
119                 NaN

In [6]:


ndf_subtype = pd.read_csv(f"../../../tascCODA_data/applications/labus_IBS/results/labus_genus_tree_agg_subtype_phi_5.csv")
ndf_subtype["Condition"] = [re.findall(r"\[.*\]", x)[0].replace("[T.", "").replace("]", "") for x in ndf_subtype["Covariate"]]
print(ndf_subtype.loc[ndf_subtype["Final Parameter"] != 0, ["Condition", "Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Final Parameter"]])
print(ndf_subtype.loc[ndf_subtype["Final Parameter"] != 0, ["Condition", "Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Final Parameter"]].to_latex(index=False))

           Condition   Kingdom        Phylum        Class  \
1              IBS-C  Bacteria  Bacteroidota  Bacteroidia   
41             IBS-C  Bacteria    Firmicutes   Clostridia   
43             IBS-C  Bacteria    Firmicutes   Clostridia   
57             IBS-C  Bacteria    Firmicutes   Clostridia   
59             IBS-C  Bacteria    Firmicutes   Clostridia   
121            IBS-D  Bacteria  Bacteroidota  Bacteroidia   
122            IBS-D  Bacteria  Bacteroidota  Bacteroidia   
242            IBS-M  Bacteria  Bacteroidota  Bacteroidia   
287            IBS-M  Bacteria    Firmicutes   Clostridia   
301            IBS-M  Bacteria    Firmicutes   Clostridia   
436  IBS-unspecified  Bacteria    Firmicutes   Clostridia   

                                   Order                 Family  \
1                          Bacteroidales         Bacteroidaceae   
41                        Lachnospirales        Lachnospiraceae   
43                        Lachnospirales        Lachnospiraceae   